## Input
`hhid`
## Output
`ranked list of cereal brands that the user most likely want to put in his/her basket`

## 1. Import modules

In [8]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

import sys
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Load data

2 data sets:
- `UPC_Hack_Cereal2.csv` contains user transactions for cereal
- `Hackathon Data Map.xlsx` contains description of each cereal brand

In [9]:
cereal = pd.read_csv('./data/national-shopper-lab/UPC_Hack_Cereal2.csv')
data_cereal = pd.read_excel('./data/national-shopper-lab/Hackathon Data Maps.xlsx', 'Cereal UPCs')

In [10]:
# format the name of columns to lowercase
def format_name(df):
    return [x.lower().strip() for x in list(df.columns)]

cereal.columns, data_cereal.columns = (df.pipe(format_name) for df in [cereal, data_cereal])

# format the date as 'YY-MM-DD'
cereal['date'] = pd.to_datetime(cereal['date'], yearfirst=True)

In [11]:
data = pd.merge(cereal, data_cereal, left_on = 'upc', right_on = 'upc', how = 'left')\
         .dropna(axis=0, how='any')\
         .drop(['store', 'upc', 'price', 'online'], axis=1)\
         .rename(columns={'qty':'purchase count','brand':'product'})
data[['hhid', 'purchase count']] = data[['hhid', 'purchase count']].astype(int)
data.sort_values('date').reset_index().drop('index', axis=1)

,hhid,date,purchase count,description,cat,parent brand,product
0,57788,2019-01-01,1,POST HBO W/ALMOND 18OZ,RTE Cereal,Post,Honey Bunches of Oats
1,157021,2019-01-01,1,KLG FRSTD MNI WHEATS BTSZ,RTE Cereal,Kelloggs,Mini Wheats
2,227987,2019-01-01,1,KLG FROOT LOOPS 1.5OZ,RTE Cereal,Kelloggs,Froot Loops
3,1019,2019-01-01,1,KELLOGG CRL RSN BRAN CRNCH,RTE Cereal,Kelloggs,Raisin Bran
4,1019,2019-01-01,1,KLG FRST FLK 13.5Z,RTE Cereal,Kelloggs,FROSTED FLAKES
...,...,...,...,...,...,...,...
6606861,1627,2021-12-31,1,NATURES PATH GRN FR MPL AMN,RTE Cereal,NATURE S PATH,NATURE S PATH
6606862,259753,2021-12-31,1,KLG FRST FLK 13.5Z,RTE Cereal,Kelloggs,FROSTED FLAKES
6606863,412956,2021-12-31,1,CRN RAIS BRN 18.2OZ,RTE Cereal,Private Label,Private Label
6606864,99350,2021-12-31,1,GM CHRIOS CRN CIN18.2Z,RTE Cereal,Big G,CHEERIOS


## 3. Data preparation

In [12]:
data = pd.merge(cereal, data_cereal, left_on = 'upc', right_on = 'upc', how = 'left')\
         .dropna(axis=0, how='any')\
         .drop(['store', 'upc', 'date', 'price', 'online', 'description', 'cat', 'parent brand'], axis=1)\
         .rename(columns={'qty':'purchase_count','brand':'product'})
data[['hhid','purchase_count']] = data[['hhid', 'purchase_count']].astype(int)
data['product'] = data['product'].apply(lambda x: x[0].upper()+x[1:].lower().strip())
print(data.shape)
data.head(10)

(6606866, 3)


,hhid,purchase_count,product
0,1,1,Nature valley
1,1,1,Kind
2,2,1,Cheerios
3,2,1,Kashi
4,2,1,Kashi
5,2,1,Kashi
6,2,1,Kashi
7,2,1,Kashi
8,2,1,Aurora natural
9,2,1,Raisin bran


## 4. Create a "sparse" matrix

- Each cell represents the purchase frequency of each user for each product
- Our goal is to *predict the values for the missing ones (NaN)*

In [13]:
data_matrix = pd.pivot_table(data, values='purchase_count', index='hhid', columns='product')
data_matrix.tail()

product,All bran,All other,Alpen,Alpina bon yurt,Annies homegrown,Apple jacks,Arrowhead mills,Aurora natural,Back to nature,Bakery on main,...,The toasted oat,Toast crunch,Total,Trix,Udis,Uncle sam,Vans,Vikis,Weetabix,Wheaties
hhid,,,,,,,,,,,,,,,,,,,,,
447537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
447538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
447539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
447541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
447542,NaN,NaN,NaN,NaN,1.0,1.0,NaN,2.0,NaN,NaN,...,NaN,1.125,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.5


We *normalize the data* with the following formula:

$x_{norm} = \frac{x-x_{min}}{x_{max}-x_{min}}$

In [14]:
data_matrix_norm = (data_matrix/data_matrix.max())
data_matrix_norm

product,All bran,All other,Alpen,Alpina bon yurt,Annies homegrown,Apple jacks,Arrowhead mills,Aurora natural,Back to nature,Bakery on main,...,The toasted oat,Toast crunch,Total,Trix,Udis,Uncle sam,Vans,Vikis,Weetabix,Wheaties
hhid,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.117647,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
447538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.017544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
447539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We create a table for input to the model

In [15]:
data_norm = data_matrix_norm.reset_index()
data_norm.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(data_norm, id_vars=['hhid'], value_name='scaled_purchase_freq').dropna()
data_norm

,hhid,product,scaled_purchase_freq
526,556,All bran,0.222222
579,613,All bran,0.166667
581,615,All bran,0.166667
582,616,All bran,0.166667
976,1034,All bran,0.166667
...,...,...,...
48544601,447205,Wheaties,0.235294
48544783,447399,Wheaties,0.117647
48544869,447496,Wheaties,0.235294
48544878,447505,Wheaties,0.117647


## 4. Split train and test set

We split the data into training and testing set with ratio of 80:20

In [16]:
def split_data(data):
    train, test = train_test_split(data, test_size = 0.2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

train_data, test_data = split_data(data_norm) # for the similarity model

## 5. Define models using Turicreate library

In [17]:
# constant variable to define field names
user_id = 'hhid'
item_id = 'product'
users_to_recommend = list(data[user_id].unique())
n_rec = 5 # number of cereal brands to recommend
n_display = 30 # to display the first few rows in an output dataset

We would recommend the 5 most relevant cereal brands to our users.

# Collaborative Filtering Model

### Decision on using Collaborative Filtering Model

Compare to the popularity based model, which only recommends the most popular items to the users based on purchase frequencie, which is non-personalized, we use a more personalized approach, which consider the user's purchase history of the items.

### Definition
- Collaborative Filtering is a technique or a method to predict a user's tatse and find the items that a user might prefer on the basis of information collected from various other users having similar tatses or preferences.
- Does not require any detail information about the product except for the purchase history of the product by the users.

### In general,
- We recommend a user items that he/she has never bought before but they were relevant to the products the user has purchased in the past.

- The two most popular forms of collaborative filtering are:
    - **User Based**: Measure the **similarity among users based on their purchase history**. We find the target user's preference of the missing item with the help of these users.
    - **Item Based**: Measure **how similar a product is to another product**. We find the target user's preference of the missing item with the help of the preference given to the other items by other users.
    
### Decision

We decide to use the **Item-to-Item Based Collborative Filtering Model** since we have more users than items.

User preferences and liking may also change over time so an item-to-item approach can tackle this problem more efficiently.

### Steps:
1. Create an item-to-item similarity matrix. This is used to measure how similar a product is to another product. We choose `cosine` similarity as our main method.
     - To compare item A and B, we look at all customers who have rated both these items. We create two item-vectors, v1 for item X, and v2 for item Y, and find the `cosine` angle/distance between these vectors. 
     - The similarity is defined by the following formula:

        + **similarity(A, B)** = $cos(\vec{A},\vec{B}) = \frac{\vec{A}.\vec{B}}{||\vec{A}||.||\vec{B}||} = \frac{\sum_u freq_{u, A}.freq_{u, B}}{\sqrt{\sum_u freq^2_{u,A}}\sqrt{\sum_u freq^2_{u, B}}}$

    - As the vectors get closer, the angle will be smaller and the `cosine` will be larger.
    - Cosine similarity gives value between -1 and 1
        - Close to 1: Two items are strongly similar to each other
        - Close to -1: Two items are strongly not similar to each other
        - CLose to 0: There is not much correlation between 2 items
        
2. Calculating the prediction of a user u for item i
    - We use the items (already rated by the user) that are most similar to the missing item to generate rating. Specifically, we try to generate predictions based on the ratings of similar products.
    - We use a formula which computes the rating for a particular item using weighted sum of the ratings of the other similar products
    
        + **prediction(u, i)** = $\frac{\sum_j (freq_{u, j}).similarity_{i,j}}{\sum_j similarity_{i,j}}$


In [18]:
cosine_similarity_model = tc.item_similarity_recommender.create(train_data, user_id=user_id, item_id=item_id, target='scaled_purchase_freq', similarity_type='cosine')
product_recommend = cosine_similarity_model.recommend(users=users_to_recommend, k=n_rec)

Preparing data set.

Data has 1638468 observations with 404549 users and 111 items.

Data prepared in: 0.661301s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 416000/429601 queries. users per second: 1.53626e+06

recommendations finished on 417000/429601 queries. users per second: 1.5344e+06

recommendations finished on 418000/429601 queries. users per second: 1.52955e+06

recommendations finished on 419000/429601 queries. users per second: 1.52527e+06

recommendations finished on 420000/429601 queries. users per second: 1.52273e+06

## 6. Final Output

In [19]:
rmse_cosine_similarity = cosine_similarity_model.evaluate_rmse(test_data, 'scaled_purchase_freq')
rmse_cosine_similarity['rmse_overall']

recommendations finished on 421000/429601 queries. users per second: 1.51853e+06

recommendations finished on 422000/429601 queries. users per second: 1.5132e+06

recommendations finished on 423000/429601 queries. users per second: 1.50879e+06

recommendations finished on 424000/429601 queries. users per second: 1.5023e+06

recommendations finished on 425000/429601 queries. users per second: 1.49839e+06

recommendations finished on 426000/429601 queries. users per second: 1.47779e+06

recommendations finished on 427000/429601 queries. users per second: 1.46648e+06

recommendations finished on 428000/429601 queries. users per second: 1.45218e+06

recommendations finished on 429000/429601 queries. users per second: 1.42532e+06

0.08026091599026114

### CSV output file

### Customer recommendation function

We select the **Cosine similarity on Purchase Dummy** approach as our final model

In [20]:
# output recommendations to csv file
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recommendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recommendation.to_dataframe()
    df_rec['recommendedBrands'] = df_rec.groupby([user_id])[item_id].transform(lambda x: ', '.join(x.astype(str)))
    df_output = df_rec[['hhid', 'recommendedBrands']].drop_duplicates().sort_values('hhid').set_index('hhid')
    if print_csv:
        df_output.to_csv('recommendation.csv')
    print('A csv file containing recommendations for users has been created')
    return df_output

df_output = create_output(cosine_similarity_model, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/429601 queries. users per second: 38457.1

recommendations finished on 2000/429601 queries. users per second: 75557.2

recommendations finished on 3000/429601 queries. users per second: 111694

recommendations finished on 4000/429601 queries. users per second: 146730

recommendations finished on 5000/429601 queries. users per second: 176685

...

recommendations finished on 427000/429601 queries. users per second: 1.51999e+06

recommendations finished on 428000/429601 queries. users per second: 1.51133e+06

recommendations finished on 429000/429601 queries. users per second: 1.489e+06

A csv file containing recommendations for users has been created
(429601, 1)


,recommendedBrands
hhid,
1,"Bear naked, Private label, Nature s path, Natu..."
2,"Kind, Private label, Nature s path, Kashi, Spe..."
3,"Cascadian farm, Private label, Nature s path, ..."
4,"Toast crunch, Private label, Special k, Froste..."
5,"Froot loops, Toast crunch, Pebbles, Lucky char..."


In [21]:
def customer_recommendation(hhid):
    if hhid not in df_output.index:
        print('Household not found.')
        return hhid
    return df_output.loc[hhid]

In [22]:
customer_recommendation(5)

recommendedBrands    Froot loops, Toast crunch, Pebbles, Lucky char...
Name: 5, dtype: object